In [633]:
import json_utils as ju
import datetime
import pandas as pd
import numpy as np
import nltk.cluster
from nltk.tokenize import RegexpTokenizer, word_tokenize
from nltk.corpus import stopwords
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
from IPython.display import clear_output

In [664]:
# Config
stops = set(stopwords.words('english'))
jsonfiles = ['chicagotribune0.json','chicagotribune1.json',\
             'chicagotribune2.json','chicagotribune3.json']
cluster_num = 3
article_keywords = ['refugee', 'refugees', 'migrant', 'migrants', 'asylum', 'rohingya',
            'immigrant', 'immigrants', 'immigration','UNHCR', 'UN Refugees',
            'deportation', 'border wall', 'illegal border crossing', 'syria','conflict'
           'rohingya']

In [665]:
# Function defs
def is_refugee_article(article_text):
    match_count = 0
    for i in article_keywords:
        if i in article_text.lower():
            match_count += 1
    if match_count >= 1:
        return True
    else:
        return False

def article_tokenize(article):
    article_clean = str(article[1:][0][:-1])
    tokenizer = RegexpTokenizer(r'[^\d\W]+')
    tokens = tokenizer.tokenize(article_clean)
    tokens = [i.lower() for i in tokens if i not in stops]
    return tokens

def extract_article_date(article):
    return article[1][-1][1]

In [658]:
# Load articles from JSON
jsons = ju.get_jsons(jsonfiles,'')
article = next(jsons)

In [659]:
# Clean articles
article_tokens = []
parsed_dates = []
for article in jsons:
    article_temp = ju.get_article_text(article['html'])
    
    # Parse failures always have length 1; 2+ otherwise
    if (len(article_temp) > 1):
        tokens = article_tokenize(article_temp)
        article_tokens.append(tokens)
        if (len(article_temp[1]) > 1):
            article_date = extract_article_date(article_temp)
            parsed_dates.append(article_date)
        else:
            parsed_dates.append('')

print("Tribune article parsing complete.")
parsed_dates = pd.to_datetime(parsed_dates)

/Users/robs/projects/hackabetterworld/social_listening/news/json_utils.py:33: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 33 of the file /Users/robs/projects/hackabetterworld/social_listening/news/json_utils.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(html)


Tribune article parsing complete.


/Users/robs/anaconda/envs/hackabetterworld/lib/python3.6/site-packages/dateutil/parser/_parser.py:1204: UnknownTimezoneWarning: tzname CDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)
/Users/robs/anaconda/envs/hackabetterworld/lib/python3.6/site-packages/dateutil/parser/_parser.py:1204: UnknownTimezoneWarning: tzname CST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


In [676]:
# Join tokens to create documents, + classify as refugee-related
text = [' '.join(i) for i in article_tokens]
refugee_class = [is_refugee_article(i) for i in text]
refugee_texts = np.array(text)[np.where(np.array(refugee_class) == True)].tolist()
text = refugee_texts

In [671]:
# Doc2Vec modeling
tagged_documents = [TaggedDocument(words=word_tokenize(_d), tags=[str(i)])\
                    for i, _d in enumerate(text)]

max_epochs = 50
vector_size = 50
alpha = 0.025

model = Doc2Vec(dm=1,
                vector_size=vector_size,
                alpha=alpha, 
                min_alpha=0.0025,
                min_count=2)
  
model.build_vocab(tagged_documents)

for epoch in range(max_epochs):
    clear_output()
    print('Training iteration ' + str(epoch+1) + ' of ' + str(max_epochs) + "...")
    model.train(tagged_documents,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha
    
model.save("news.model")

Training iteration 50 of 50...


In [486]:
# Measure distance between documents
document_vectors = []
for i in range(0,len(article_tokens)):
    vec = model.infer_vector(text[i])
    document_vectors.append(vec)

In [ ]:
# Generate clusters
kmeans = nltk.cluster.KMeansClusterer(cluster_num, avoid_empty_clusters=True,\
                                          distance=nltk.cluster.util.cosine_distance,\
                                          repeats=20)
clusters = kmeans.cluster(document_vectors, assign_clusters=True)
cluster_counts = pd.Series(clusters).value_counts()

In [586]:
# Describe clusters by most frequent words
top_term_list = []
for i in range(len(cluster_counts)):
    docs = np.where(np.array(clusters) == i)[0]
    terms = np.asarray(article_tokens)[docs]
    term_counts = pd.Series([item for sublist in terms for item in sublist]).value_counts()
    top_terms = term_counts.sort_values(ascending=False)[:50]
    top_term_list.append(top_terms)
    

In [521]:
# Cluster time series
clusterDF = pd.DataFrame({'Cluster': clusters, 'Time': parsed_dates,
                         'Month': parsed_dates.to_period('M'),
                          'Week': parsed_dates.to_period('W'),
                         'Article Count': 1})
clusterDF.sort_values(by='Time',inplace=True)
clusterDF.set_index('Time',inplace=True)
cluster_time_series = clusterDF.groupby(['Week','Cluster']).sum()